In [3]:
from threading import Thread
from math import pi

import rclpy
from rclpy.callback_groups import ReentrantCallbackGroup
from rclpy.node import Node

from pymoveit2 import MoveIt2
from pymoveit2.robots import ur

from geometry_msgs.msg import Pose, Quaternion, Point
import rclpy.wait_for_message
from ur_msgs.srv import SetIO
from ur_commander.srv import VisualizePoses

from tf_transformations import quaternion_multiply, quaternion_from_euler
import numpy as np
from geometry_msgs.msg import PoseStamped, Pose, Point, Quaternion
from scipy.spatial.transform import Rotation

rclpy.init()

# Create node for this example
node = Node("notebook_example")
callback_group = ReentrantCallbackGroup()


def display_poses(poses: list[Pose], frame_id: str = "ur_base_link") -> None:
    client = node.create_client(VisualizePoses, "/visualize_poses")
    while not client.wait_for_service(timeout_sec=1.0):
        node.get_logger().info("service not available, waiting again...")
    client.call(VisualizePoses.Request(poses=poses, frame_id=frame_id))


moveit2 = MoveIt2(
    node=node,
    joint_names=ur.joint_names(),
    base_link_name=ur.base_link_name(),
    end_effector_name=ur.end_effector_name(),
    group_name=ur.MOVE_GROUP_ARM,
    callback_group=callback_group,
)

# Spin the node in background thread(s) and wait a bit for initialization
executor = rclpy.executors.MultiThreadedExecutor(2)
executor.add_node(node)
executor_thread = Thread(target=executor.spin, daemon=True, args=())
executor_thread.start()
node.create_rate(1.0).sleep()

# Scale down velocity and acceleration of joints (percentage of maximum)
moveit2.max_velocity = 0.1
moveit2.max_acceleration = 0.1
synchronous = True
cancel_after_secs = 0.0
cartesian = True
cartesian_max_step = 0.01
cartesian_fraction_threshold = 0.8
cartesian_jump_threshold = 0.0
cartesian_avoid_collisions = True
moveit2.planner_id = "RRTkConfigDefaul"

# # Add collision objects
# moveit2.add_collision_box(
#     id="table",
#     size=[2.0, 1.0, 0.05],
#     position=[0.0, 0.0, -0.025],
#     quat_xyzw=[0.0, 0.0, 0.0, 0.0],
# )





AttributeError: module 'numpy' has no attribute 'float'.
`np.float` was a deprecated alias for the builtin `float`. To avoid this error in existing code, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [2]:
husky_size = {"x":0.99, "y":0.67, "z":0.39}
boxes = {"x":0.325, "y":0.49, "z":0.40}
ground_height = -husky_size["z"] - 0.01/2 - 0.05

print(boxes["x"])

# Add collision objects
moveit2.add_collision_box(
    id="husky",
    size=[husky_size["x"] + 0.05, husky_size["y"] + 0.05, husky_size["z"] + 0.05],
    position=[-0.3, 0.0, -husky_size["z"]/2 - 0.05],
    quat_xyzw=[0.0, 0.0, 0.0, 0.0],
)

moveit2.add_collision_box(
    id="controller",
    size=[0.45, 0.65, 0.7],
    position=[-0.7, 0.0, 0.34],
    quat_xyzw=[0.0, 0.0, 0.0, 0.0],
)

moveit2.add_collision_box(
    id="floor",
    size=[4.0, 4.0, 0.01],
    position=[0.0, 0.0, -husky_size["z"] - 0.01/2 - 0.05],
    quat_xyzw=[0.0, 0.0, 0.0, 0.0],
)

0.325


**Define a Target Pose**    

Display the pose

In [20]:
home_position = [0.0, -pi / 2, pi / 2, 0.0, 0.0, 0.0]
node.get_logger().info("Moving to homecartesian =True position")
traj_home = moveit2.move_to_configuration(home_position)
if traj_home is None:home_position = [0.0, -pi / 2, pi / 2, 0.0, 0.0, 0.0]
node.get_logger().info("Moving to homecartesian =True position")
traj_home = moveit2.move_to_configuration(home_position)
if traj_home is None:
    node.get_logger().error("Failed to move to home position")
    node.get_logger().error("Failed to move to home position")

moveit2.execute(traj_home)
success = moveit2.wait_until_executed()
if not success:
    node.get_logger().error("Failed to execute trajectory")
else:
    node.get_logger().info("Moved to home position")

[INFO] [1741793359.767038795] [notebook_example]: Moving to homecartesian =True position
[INFO] [1741793359.889363754] [notebook_example]: Moving to homecartesian =True position
[INFO] [1741793383.962625926] [notebook_example]: Moved to home position


In [3]:
home = Pose()

home.position.x = 0.6
home.position.y = 0.1
home.position.z = 0.7

h = quaternion_from_euler(-pi*3/4, 0.0, 0.0)

home.orientation.x = h[0]
home.orientation.y = h[1]
home.orientation.z = h[2]
home.orientation.w = h[3]

display_poses([home])

traj = moveit2.move_to_pose(
                    position=home.position,
                    quat_xyzw=home.orientation,
                    cartesian=cartesian,
                    cartesian_max_step=cartesian_max_step,
                    cartesian_fraction_threshold=cartesian_fraction_threshold,
                )

if traj is not None:
    node.get_logger().info(f"Move to target {home}")
    moveit2.execute(traj)
    success = moveit2.wait_until_executed()
    print(success)

[INFO] [1741793694.189868577] [notebook_example]: Move to target geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.6, y=0.1, z=0.7), orientation=geometry_msgs.msg.Quaternion(x=-0.9238795325112867, y=0.0, z=0.0, w=0.38268343236508984))


True


In [ ]:
if traj is not None:
    node.get_logger().info(f"Move to target {home}")
    moveit2.execute(traj)
    success = moveit2.wait_until_executed()
    print(success)

In [6]:

collision_object_location = Pose()

collision_object_location.position.x = boxes["x"]/2 + 0.42
collision_object_location.position.y = 0.0
collision_object_location.position.z = ground_height + boxes["z"]/2

collision_object_location.orientation = Quaternion(x=0.0, y=0.0, z=0.0, w=1.0)

moveit2.add_collision_box(
    id="box",
    size=[boxes["x"], boxes["y"], boxes["z"]],
    position=collision_object_location.position,
    quat_xyzw=collision_object_location.orientation,
)



box_location = Pose()

box_location.position = collision_object_location.position

b1 = quaternion_from_euler(-pi*3/4, 0.0, 0.0)

box_location.orientation.x = b1[0]
box_location.orientation.y = b1[1]
box_location.orientation.z = b1[2]
box_location.orientation.w = b1[3]


print(box_location.position.x)



start = Pose()

start.position.x = box_location.position.x
start.position.y = -(box_location.position.y + boxes["y"]/2 + 0.1)
start.position.z = box_location.position.z + 0.05

start.orientation = box_location.orientation


approch = Pose()

approch.position.x = start.position.x 
approch.position.y = start.position.y
approch.position.z = start.position.z + 0.3

approch.orientation = start.orientation




stop = Pose()

stop.position.x = box_location.position.x
stop.position.y = box_location.position.y + husky_size["y"]/2 + 0.1
stop.position.z = box_location.position.z + 0.05

stop.orientation = start.orientation

display_poses([box_location,approch, start, stop])

safe_stop = Pose()

safe_stop.position.x = stop.position.x
safe_stop.position.y = stop.position.y
safe_stop.position.z = stop.position.z + 0.3

safe_stop.orientation = stop.orientation



print(start.position)
display_poses([box_location, approch, start, stop, safe_stop])




0.5825
geometry_msgs.msg.Point(x=0.5825, y=-0.345, z=-0.195)


In [ ]:
print(help(moveit2))

In [ ]:
targets = [approch, start,stop, safe_stop]

In [ ]:
cartesian =True

In [24]:
traj = moveit2.move_to_pose(
    position=approch.position,
    quat_xyzw=approch.orientation,
    cartesian=True,
    cartesian_max_step=cartesian_max_step,
    cartesian_fraction_threshold=cartesian_fraction_threshold,
)

In [25]:
if traj is not None:
    node.get_logger().info(f"Move to target {approch}")
    moveit2.execute(traj)
    success = moveit2.wait_until_executed()
    print(success)

[INFO] [1741793409.746291808] [notebook_example]: Move to target geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.5825, y=-0.345, z=0.10499999999999998), orientation=geometry_msgs.msg.Quaternion(x=-0.9238795325112867, y=0.0, z=0.0, w=0.38268343236508984))


True


In [26]:
traj = moveit2.move_to_pose(
    position=start.position,
    quat_xyzw=start.orientation,
    cartesian=True,
    cartesian_max_step=cartesian_max_step,
    cartesian_fraction_threshold=cartesian_fraction_threshold,
)

In [27]:
if traj is not None:
    node.get_logger().info(f"Move to target {start}")
    moveit2.execute(traj)
    success = moveit2.wait_until_executed()
    print(success)
    moveit2.remove_collision_object("box")

[INFO] [1741793414.638586689] [notebook_example]: Move to target geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.5825, y=-0.345, z=-0.195), orientation=geometry_msgs.msg.Quaternion(x=-0.9238795325112867, y=0.0, z=0.0, w=0.38268343236508984))


True


In [28]:
traj = moveit2.move_to_pose(
    position=stop.position,
    quat_xyzw=stop.orientation,
    cartesian=cartesian,
    cartesian_max_step=cartesian_max_step,
    cartesian_fraction_threshold=cartesian_fraction_threshold,
)

In [29]:
if traj is not None:
    node.get_logger().info(f"Move to target {stop}")
    moveit2.execute(traj)
    success = moveit2.wait_until_executed()
    print(success)

[INFO] [1741793418.818124797] [notebook_example]: Move to target geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.5825, y=0.43500000000000005, z=-0.195), orientation=geometry_msgs.msg.Quaternion(x=-0.9238795325112867, y=0.0, z=0.0, w=0.38268343236508984))


True


In [30]:
traj = moveit2.move_to_pose(
    position=safe_stop.position,
    quat_xyzw=safe_stop.orientation,
    cartesian=True,
    cartesian_max_step=cartesian_max_step,
    cartesian_fraction_threshold=cartesian_fraction_threshold,
)

In [31]:
if traj is not None:
    node.get_logger().info(f"Move to target {safe_stop}")
    moveit2.execute(traj)
    success = moveit2.wait_until_executed()
    print(success)

[INFO] [1741793423.514305693] [notebook_example]: Move to target geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.5825, y=0.43500000000000005, z=0.10499999999999998), orientation=geometry_msgs.msg.Quaternion(x=-0.9238795325112867, y=0.0, z=0.0, w=0.38268343236508984))


True


In [17]:
home_position = [0.0, -pi / 2, pi / 2, 0.0, 0.0, 0.0]
node.get_logger().info("Moving to homecartesian =True position")
traj_home = moveit2.move_to_configuration(home_position)
if traj_home is None:
    node.get_logger().error("Failed to move to home position")

[INFO] [1741793263.106066621] [notebook_example]: Moving to homecartesian =True position


In [16]:
moveit2.execute(traj_home)
success = moveit2.wait_until_executed()
if not success:
    node.get_logger().error("Failed to execute trajectory")
else:
    node.get_logger().info("Moved to home position")

[INFO] [1741792434.037388338] [notebook_example]: Moved to home position


In [ ]:
home = Pose()

home.position.x = 0.6
home.position.y = 0.1
home.position.z = 0.7

h = quaternion_from_euler(-pi*3/4, 0.0, 0.0)

home.orientation.x = h[0]
home.orientation.y = h[1]
home.orientation.z = h[2]
home.orientation.w = h[3]

display_poses([home])

traj = moveit2.move_to_pose(
                    position=home.position,
                    quat_xyzw=home.orientation,
                    cartesian=cartesian,
                    cartesian_max_step=cartesian_max_step,
                    cartesian_fraction_threshold=cartesian_fraction_threshold,
                )

if traj is not None:
    node.get_logger().info(f"Move to target {home}")
    moveit2.execute(traj)
    success = moveit2.wait_until_executed()
    print(success)

In [7]:
targets = [approch, start,stop, safe_stop,home]
current_task = 0
cartesian = True

for k in targets:

    if current_task == 1:
        moveit2.remove_collision_object("box")


    traj = moveit2.move_to_pose(
        position=k.position,
        quat_xyzw=k.orientation,
        cartesian=True,
        cartesian_max_step=cartesian_max_step,
        cartesian_fraction_threshold=cartesian_fraction_threshold,
    )

    if traj is not None:
        node.get_logger().info(f"Move to target {current_task}")
        moveit2.execute(traj)
        success = moveit2.wait_until_executed()
        print(success)
        current_task += 1
        
    if current_task == len(targets):
        current_task = 0
        break


[INFO] [1741793835.274223916] [notebook_example]: Move to target 0


True


[INFO] [1741793843.166599846] [notebook_example]: Move to target 1


True


[INFO] [1741793848.760779766] [notebook_example]: Move to target 2


True


[INFO] [1741793858.448457850] [notebook_example]: Move to target 3


True


[INFO] [1741793864.232007420] [notebook_example]: Move to target 4


True


In [ ]:
home = Pose()

home.position.x = 0.6
home.position.y = 0.1
home.position.z = 0.7

h = quaternion_from_euler(-pi*3/4, 0.0, 0.0)

home.orientation.x = h[0]
home.orientation.y = h[1]
home.orientation.z = h[2]
home.orientation.w = h[3]

display_poses([home])

traj = moveit2.move_to_pose(
                    position=home.position,
                    quat_xyzw=home.orientation,
                    cartesian=cartesian,
                    cartesian_max_step=cartesian_max_step,
                    cartesian_fraction_threshold=cartesian_fraction_threshold,
                )

if traj is not None:
    node.get_logger().info(f"Move to target {home}")
    moveit2.execute(traj)
    success = moveit2.wait_until_executed()
    print(success)

In [ ]:
# Add collision objects
length= 0.070
width= 0.030
height= 0.018

moveit2.add_collision_box(
    id="object",
    size=[length, width, height],
    position=[target.position.x, target.position.y, (target.position.z - height/2)],
    quat_xyzw=target.orientation,
)


In [ ]:
approach_target = Pose(
    position=Point(
            x=pose_subscriber.latest_pose.position.x,
            y=pose_subscriber.latest_pose.position.y,
            z=pose_subscriber.latest_pose.position.z + 0.2
        ),
    orientation=target.orientation
        )
if approach_target is not None:
    display_poses([approach_target])

Plan a trajectory

In [ ]:
position = approach_target.position
quat_xyzw = approach_target .orientation


node.get_logger().info(f"Moving to approach_target")
traj = moveit2.move_to_pose(
    position=position,
    quat_xyzw=quat_xyzw,
    cartesian=True,
    cartesian_max_step=cartesian_max_step,
    cartesian_fraction_threshold=cartesian_fraction_threshold,
)

Execute the trajectory

In [ ]:
moveit2.execute(traj)
success = moveit2.wait_until_executed()
if not success:
    node.get_logger().error("Failed to execute trajectory")

In [ ]:
position = [target.position.x, target.position.y, target.position.z]
quat_xyzw = [target.orientation.x, target.orientation.y, target.orientation.z, target.orientation.w]


node.get_logger().info(f"Moving to {{position: {list(position)}, quat_xyzw: {list(quat_xyzw)}}}")
traj = moveit2.move_to_pose(
    position=position,
    quat_xyzw=quat_xyzw,
    cartesian=True,
    cartesian_max_step=cartesian_max_step,
    cartesian_fraction_threshold=cartesian_fraction_threshold,
)

In [ ]:
moveit2.execute(traj)
success = moveit2.wait_until_executed()
if not success:
    node.get_logger().error("Failed to execute trajectory")

In [ ]:
safe_out = Pose(
    position=Point(
            x=pose_subscriber.latest_pose.position.x,
            y=pose_subscriber.latest_pose.position.y,
            z=pose_subscriber.latest_pose.position.z + 0.2
        ),
    orientation=target.orientation
        )
if approach_target is not None:
    display_poses([approach_target])

In [ ]:
position = approach_target.position
quat_xyzw = approach_target .orientation


node.get_logger().info(f"Moving to safe_out")
traj = moveit2.move_to_pose(
    position=position,
    quat_xyzw=quat_xyzw,
    cartesian=True,
    cartesian_max_step=cartesian_max_step,
    cartesian_fraction_threshold=cartesian_fraction_threshold,
)


In [ ]:
moveit2.execute(traj)
success = moveit2.wait_until_executed()
if not success:
    node.get_logger().error("Failed to execute trajectory")

In [ ]:

place_palne = Pose(position=Point(x=0.3, y=0.3, z=(height/2)),
                   orientation=target.orientation)
display_poses([place_palne])
